In [1]:
# base imports
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm
import pickle
import copy
from time import time
import matplotlib.pyplot as plt

%config IPCompleter.greedy = True

In [2]:
# Data preparation: 
# Some data is missing (but we know what it should be) or some data should be changed.
df = pd.read_csv('./speed_dating_data.csv', encoding = 'ISO-8859-1')

def fooCareer(col):
    selected_df = df[col]
    career_c = {
        'lawyer': 1.,
        'law': 1.,
        'Economics': 7.,
        'tech professional': 5.
    }
    
    for x, row in selected_df.iterrows():
            career = df.at[x, col[0]]
            if career in career_c:
                df.at[x, col[1]] = career_c[career]
                

def fooGoal(col):
    selected_df = df[col]
    goal_c = {
        1: 1,
        2: 1,
        3: 1,
        4: 0,
        5: 1,
        6: 1,
    }
    
    for x, row in selected_df.iteritems():
        goal = df.at[x, col]
        if goal in goal_c:
            df.at[x, col] = goal_c[goal]

fooCareer(['career','career_c'])
fooGoal('goal')
df.to_csv('base_dating_data.csv', index=False)

In [3]:
# data is inconsistent:
# Some values (it’s based on waves) are 1-10 and others are 1-100 so
# they should be normalized so that all are 1-100.
df = pd.read_csv('base_dating_data.csv')

def func(lista):
    selected_df = df[lista]
    for x, row in selected_df.iterrows():
        multiplier = 100/row.sum()
        for col in lista:
            df.at[x, col] *= multiplier

lista = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
lista1 = ['attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o']
lista3 = ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
lista4 = ['attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2']

func(lista)
func(lista1)
func(lista3)
func(lista4)

df.to_csv('base_dating_data.csv', index=False)

<ipython-input-3-023d9bb5a582>:9: RuntimeWarning: divide by zero encountered in double_scalars
  multiplier = 100/row.sum()
<ipython-input-3-023d9bb5a582>:11: RuntimeWarning: invalid value encountered in double_scalars
  df.at[x, col] *= multiplier


In [4]:
# Column met_o had values 1/2. It needs to be changed to 0/1.
df = pd.read_csv('./base_dating_data.csv')
df['met_o'] = df['met_o'].apply(lambda x: x - 1)
df.to_csv('base_dating_data.csv', index=False)

We removed some rows (by hand) because they didn't have `pid`.

### Below you can see code that prepares data for the basic matrix factorization.
Here in the base matrix we only have information about match. So we need a set of vectors, where each vector describes each date (holds both ids and information about match).

Task will requite two base matrices.
1. Matrix where men are "users" and women are "products". It will then be used to recommend women to men because matrix will say what's the predicted rating of a woman in eyes of man. Basically it will answer the question: **"How likely is that a man will like a woman?"**. Let's call this matrix/data frame **"men_like_women"**.
1. Matrix where women are "users" and men are "products". It will then be used to recommend men to women because matrix will say what's the predicted rating of a man in eyes of woman. Basically it will answer the question: **"How likely is that a woman will like a man?"**. Let's call this matrix/data frame **"women_like_men"**.

Why such analogies? It may help to understand how do this human relations task translates into recommender systems world.

These matrices will be used to train models (with matrix factorization) which will then be saved into csv files.

In [2]:
# Split into vectors, let's have two matrices as describes above.
base_df = pd.read_csv('./data_for_matrix.csv')
men_like_women_data = []
women_like_men_data = []

for _, row in base_df.iterrows():
    vector = {
        'id': row['iid'],
        'pid': row['pid'],
        'decision': row['dec'],
    }
    if row['gender'] == 0:
        # it's a woman
        women_like_men_data.append(vector)
    else:
        men_like_women_data.append(vector)

men_like_women_df = pd.DataFrame(men_like_women_data)
women_like_men_df = pd.DataFrame(women_like_men_data)

print("men_like_women_df:")
print(men_like_women_df)
print("\nwomen_like_men_df:")
print(women_like_men_df)

men_like_women_df:
         id    pid  decision
0      11.0    1.0       0.0
1      11.0    2.0       0.0
2      11.0    3.0       0.0
3      11.0    4.0       0.0
4      11.0    5.0       0.0
...     ...    ...       ...
4179  552.0  526.0       0.0
4180  552.0  527.0       0.0
4181  552.0  528.0       0.0
4182  552.0  529.0       0.0
4183  552.0  530.0       0.0

[4184 rows x 3 columns]

women_like_men_df:
         id    pid  decision
0       1.0   11.0       1.0
1       1.0   12.0       1.0
2       1.0   13.0       1.0
3       1.0   14.0       1.0
4       1.0   15.0       1.0
...     ...    ...       ...
4179  530.0  548.0       0.0
4180  530.0  549.0       1.0
4181  530.0  550.0       0.0
4182  530.0  551.0       0.0
4183  530.0  552.0       1.0

[4184 rows x 3 columns]


## Let's prepare train and test data sets

### Make ids contiguous
We need to change the data. As you can see ids are not contiguous and they must be. So let's change that.

In [3]:
def proc_col(column):
    """Encodes a pandas column with continous ids."""
    unique = column.unique()
    old_to_new = {o: i for i, o in enumerate(unique)}
    return np.array([old_to_new.get(x, -1) for x in column])


def encode_data(df):
    """Encodes data with continous person and partner ids."""
    df = df.copy()
    for col_name in ["id", "pid"]:
        col = proc_col(df[col_name])
        df[col_name] = col
        df = df[df[col_name] >= 0]
        
    return df

And now use these functions to make ids contiguous.

In [4]:
print('Before:')
print(men_like_women_df)
men_like_women_df = encode_data(men_like_women_df)
print('\nAfter:')
print(men_like_women_df)

women_like_men_df = encode_data(women_like_men_df)

Before:
         id    pid  decision
0      11.0    1.0       0.0
1      11.0    2.0       0.0
2      11.0    3.0       0.0
3      11.0    4.0       0.0
4      11.0    5.0       0.0
...     ...    ...       ...
4179  552.0  526.0       0.0
4180  552.0  527.0       0.0
4181  552.0  528.0       0.0
4182  552.0  529.0       0.0
4183  552.0  530.0       0.0

[4184 rows x 3 columns]

After:
       id  pid  decision
0       0    0       0.0
1       0    1       0.0
2       0    2       0.0
3       0    3       0.0
4       0    4       0.0
...   ...  ...       ...
4179  276  269       0.0
4180  276  270       0.0
4181  276  271       0.0
4182  276  272       0.0
4183  276  273       0.0

[4184 rows x 3 columns]


### Let's split into train and test data sets
Standard sklearn function `train_test_split` doesn't do a job here because both test and train data sets should include all people. So after using `train_test_split` we need to transition some people between sets to ensure that both sets are correct. The same goes for partners.

In [5]:
from sklearn.model_selection import train_test_split as tts

def train_test_split(df, test_size=0.2):
    # Use standard train_test_split.
    df_train, df_test = tts(df, test_size=test_size)
    no_train_unique = len(df_train.id.unique())
    no_test_unique = len(df_test.id.unique())
    
    # See what people are missing in the test set.
    diff = np.setdiff1d(df_train.id.unique(), df_test.id.unique())
    for id in diff:
        # For every missing person we need to exchange them for a
        # different one so that sets still have the same number of elements.
        person_to_send_to_test = df_train.loc[df_train.id == id].iloc[0]
        person_to_send_to_train = None
        # Choose some person from the test set to send to the train set.
        ids = df_test.id.unique()
        np.random.shuffle(ids)
        for test_id in ids:
            person = df_test.loc[df_test.id == test_id]
            if len(person.index) > 1:
                person_to_send_to_train = person.iloc[0]
                break
        if person_to_send_to_train is not None:
            # Remove people that transit from old sets.
            # .name holds the id of that row in the oryginal df.
            df_train = df_train.drop(person_to_send_to_test.name)
            df_test = df_test.drop(person_to_send_to_train.name)
            # Add new people to sets.
            df_train = pd.concat([df_train, person_to_send_to_train.to_frame().T], ignore_index=True)
            df_test = pd.concat([df_test, person_to_send_to_test.to_frame().T], ignore_index=True)
        else:
            raise Exception("Couldn't find any person from people to send from the test to the train.")
        
    # See what partners are missing in the test set.
    diff = np.setdiff1d(df_train.pid.unique(), df_test.pid.unique())
    for pid in diff:
        # For every missing partner we need to exchange them for a
        # different one so that sets still have the same number of elements.
        partner_to_send_to_test = df_train.loc[df_train.pid == pid].iloc[0]
        partner_to_send_to_train = None
        
        # Choose some partner from the test set to send to the train set.
        # Need to make sure that both sets will still have all the people.
        pids = df_test.pid.unique()
        np.random.shuffle(pids)
        for test_pid in pids:
            partner = df_test.loc[df_test.pid == test_pid]
            if len(partner.index) > 1:
                # Make sure we don't remove a person completely.
                id = partner.iloc[0].id
                person_qty = len(df_test.loc[df_test.id == id].index)
                if person_qty > 1:
                    partner_to_send_to_train = partner.iloc[0]
                    break
        if partner_to_send_to_train is not None:
            # Remove partners that transit from old sets.
            df_train = df_train.drop(partner_to_send_to_test.name)
            df_test = df_test.drop(partner_to_send_to_train.name)
            # Add new people to sets.
            df_train = pd.concat([df_train, partner_to_send_to_train.to_frame().T], ignore_index=True)
            df_test = pd.concat([df_test, partner_to_send_to_test.to_frame().T], ignore_index=True)
        else:
            raise Exception("Couldn't find any partner from partners to send from the test to the train.")
                    
    df_train = df_train.sort_values(by='id')
    df_test = df_test.sort_values(by='id')
    return df_train.reset_index(drop=True), df_test.sort_values(by='id').reset_index(drop=True)

### And now just get correct data

In [6]:
men_like_women_train_df, men_like_women_test_df = train_test_split(men_like_women_df, test_size=0.2)
men_like_women_no_men = len(men_like_women_train_df.id.unique())
men_like_women_no_women = len(men_like_women_train_df.pid.unique())

women_like_men_train_df, women_like_men_test_df = train_test_split(women_like_men_df, test_size=0.2)
women_like_men_no_women = len(women_like_men_train_df.id.unique())
women_like_men_no_men = len(women_like_men_train_df.pid.unique())

## Let's create matrix factorization models

We will create and train several base MF (matrix factorization) models and for each of them let's do a cross validation to learn the best hyperparameters and parameters. Then we will compare the results and choose the best model.

Some general explanations for models:
* Models are train on only one batch because our data set is rather small.

Good reading resource: https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cuda')

### Matrix factorization

In [21]:
class MatrixFactorizationWithoutXavier(nn.Module):
    def __init__(self, num_people, num_partners, weights=(0, 1), bias=(-0.01, 0.01), emb_size=100):
        super(MatrixFactorizationWithoutXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.person_bias = nn.Embedding(num_people, 1)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.parnter_bias = nn.Embedding(num_partners, 1)
        self.person_emb.weight.data.uniform_(weights[0], weights[1])
        self.partner_emb.weight.data.uniform_(weights[0], weights[1])
        self.person_bias.weight.data.uniform_(bias[0], bias[1])
        self.parnter_bias.weight.data.uniform_(bias[0], bias[1])
            
    def forward(self, u, v):
        bias_u = self.person_bias(u).squeeze()
        bias_v = self.parnter_bias(v).squeeze()
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1) + bias_u + bias_v)
    
    
class MatrixFactorizationWithXavier(nn.Module):
    def __init__(self, num_people, num_partners, bias=(-0.01, 0.01), emb_size=100):
        super(MatrixFactorizationWithXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.person_bias = nn.Embedding(num_people, 1)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.parnter_bias = nn.Embedding(num_partners, 1)
        torch.nn.init.xavier_uniform_(self.person_emb.weight)
        torch.nn.init.xavier_uniform_(self.partner_emb.weight)
        self.person_bias.weight.data.uniform_(bias[0], bias[1])
        self.parnter_bias.weight.data.uniform_(bias[0], bias[1])
            
    def forward(self, u, v):
        bias_u = self.person_bias(u).squeeze()
        bias_v = self.parnter_bias(v).squeeze()
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1) + bias_u + bias_v)
    

# Example small models demonstrating weights
example_model_no_xavier = MatrixFactorizationWithoutXavier(10, 10, bias=(0, 0), emb_size=3)
example_model_xavier = MatrixFactorizationWithXavier(10, 10, emb_size=3)
print("Model with without xavier weights are:\n")
for p in example_model_no_xavier.parameters():
    print(p)
print('\n\n', '='*20)
print("\n\nModel with with xavier weights are:\n")
for p in example_model_xavier.parameters():
    print(p)

Model with without xavier weights are:

Parameter containing:
tensor([[0.5521, 0.7316, 0.6421],
        [0.4934, 0.1468, 0.3583],
        [0.8805, 0.8409, 0.1527],
        [0.6849, 0.8745, 0.9804],
        [0.8551, 0.0748, 0.0664],
        [0.9288, 0.1801, 0.9321],
        [0.1513, 0.5629, 0.7011],
        [0.8393, 0.8449, 0.0659],
        [0.0889, 0.7630, 0.2972],
        [0.8641, 0.1405, 0.9920]], requires_grad=True)
Parameter containing:
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], requires_grad=True)
Parameter containing:
tensor([[9.8606e-02, 4.6385e-01, 9.0970e-01],
        [4.9359e-04, 1.4748e-01, 2.0012e-01],
        [9.7149e-01, 5.1709e-01, 5.3811e-01],
        [5.7332e-01, 4.4819e-01, 1.4420e-01],
        [3.2387e-01, 5.5951e-01, 6.4002e-01],
        [1.2382e-01, 6.0092e-01, 7.7423e-01],
        [6.2060e-01, 1.0317e-01, 4.8530e-01],
        [7.5273e-01, 5.5194e-01, 4.3380e-03],
    

### Training and testing functions are below

In [23]:
def MF_test(model, df_test, verbose=False):
    model.eval()
    # .to(dev) puts code on either gpu or cpu.
    people = torch.LongTensor(df_test.id.values).to(dev)
    partners = torch.LongTensor(df_test.pid.values).to(dev)
    decision = torch.FloatTensor(df_test.decision.values).to(dev)
    y_hat = model(people, partners)
    loss = F.mse_loss(y_hat, decision)
    if verbose:
        print('test loss %.3f ' % loss.item())
    return loss.item()


# Default values assigned below are ones that I found online.
# Cross validadtion will be done later but it's good to have some defaults.
def MF_train(model, df_train, epochs=100, learning_rate=0.01, weight_decay=1e-5, verbose=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epochs):
        # From numpy to PyTorch tensors.
        # .to(dev) puts code on either gpu or cpu.
        people = torch.LongTensor(df_train.id.values).to(dev)
        partners = torch.LongTensor(df_train.pid.values).to(dev)
        decision = torch.FloatTensor(df_train.decision.values).to(dev)
        
        # calls forward method of the model
        y_hat = model(people, partners)
        # Using mean squared errors loss function
        loss = F.mse_loss(y_hat, decision)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if verbose and epoch % 100 == 0: 
            print(loss.item())

# Let's finally train models and choose the best one

## Cross validation on each model

In [24]:
def cross_valid_model(model, train_df, test_df):
    """Function to choose the best hyperparameters for a model."""
    min_loss = np.inf
    best_settings = None

    for (epochs, wd, lr) in product(epochs_poss, weight_decay_poss, learning_rate_poss):
        MF_train(model, train_df, epochs=epochs, learning_rate=lr, weight_decay=wd)
        test_loss = MF_test(model, test_df)
        if (test_loss < min_loss) or (test_loss == min_loss and best_settings is not None and epochs < best_settings['epochs']):
            min_loss = test_loss
            best_settings = {'epochs': epochs, 'weight_decay': wd, 'learning_rate': lr}
    return min_loss, best_settings


def cross_valid(
    num_people, num_partners, train_df, test_df,
    epochs_poss,
    weight_decay_poss,
    learning_rate_poss,
    weights_poss,
    bias_poss,
    emb_size_poss,
    verbose=False, file=None
):
    """
    Function to choose the best model.
    If arg file is provided (should be a path), statistics will be written to the given file.
    """
    min_loss = np.inf
    best_settings = None
    models = []
    start_time = time()
    
    for (weights, bias, emb_size) in tqdm(product(weights_poss, bias_poss, emb_size_poss)):
        model = None
        if weights == 'xavier':
            model = MatrixFactorizationWithXavier(num_people, num_partners, bias=bias, emb_size=emb_size).to(dev)
        else:
            model = MatrixFactorizationWithoutXavier(num_people, num_partners, weights=weights, bias=bias, emb_size=emb_size).to(dev)
        
        model_min_loss, model_best_settings = cross_valid_model(model, train_df, test_df)
        
        curr_settings = {'model': model.__class__.__name__, 'min_loss': model_min_loss, 'weights': weights, 'bias': bias, 'emb_size': emb_size}
        model_best_settings = {**curr_settings, **model_best_settings}
        models.append(model_best_settings)
            
        if verbose:
            print(f'best_settings: {model_best_settings}')
            print(f'{"="*50}')
        
        if (model_min_loss < min_loss) or (model_min_loss == min_loss and best_settings and emb_size < best_settings['emb_size']):
            min_loss = model_min_loss
            best_settings = model_best_settings
    
    end_time = time()
        
    if verbose:
        print(f'\n{"?"*50}')
        print('THE WINNER IS:')
        print(f'best min_loss: {min_loss}')
        print(f'best best_settings: {best_settings}')
        print(f'Cross validation took {end_time - start_time}')
    
    if file:
        models.sort(key=lambda x: x['min_loss'])
        with open(file, 'wb') as stats_file:
            pickle.dump(models, stats_file)
        
    return min_loss, best_settings

## Let's first train women_like_man

In [33]:
MF_CROSS_VALID_RES_FILE = './women_like_men_MF_cross_vaild_results.txt'

In [35]:
epochs_poss = list(range(60, 350, 10))
weight_decay_poss = [1e-5, 0.0001, 0.001, 0.01]
learning_rate_poss = [1e-5, 0.0001, 0.001, 0.01]
weights_poss = [(0, 1), (-1, 1), (0, 0.2), (-0.2, 0.2), 'xavier']
bias_poss = [(0, 0), (0, 1), (-1, 1)]
emb_size_poss = list(range(300, 1100, 100))

min_loss, best_settings = cross_valid(
    women_like_men_no_women, women_like_men_no_men,
    women_like_men_train_df, women_like_men_test_df,
    epochs_poss=epochs_poss,
    weight_decay_poss=weight_decay_poss,
    learning_rate_poss=learning_rate_poss,
    weights_poss=weights_poss,
    bias_poss=bias_poss,
    emb_size_poss=emb_size_poss,
    file=MF_CROSS_VALID_RES_FILE
)

120it [8:47:18, 263.65s/it]


Below you can see results of the above cross valid. Models are sorted.

In [36]:
with open(MF_CROSS_VALID_RES_FILE, 'rb') as file:
    mf_cross_valid_results = pickle.load(file)
            
# sorting
mf_cross_valid_results.sort(key=lambda x: x['min_loss'])
mf_cv_res_pd = pd.DataFrame(mf_cross_valid_results)
mf_cv_res_pd

,model,min_loss,weights,bias,emb_size,epochs,weight_decay,learning_rate
0,MatrixFactorizationWithoutXavier,0.178992,"(0, 0.2)","(0, 0)",1000,250,0.001,0.0001
1,MatrixFactorizationWithoutXavier,0.179021,"(-1, 1)","(0, 1)",1000,250,0.001,0.0001
2,MatrixFactorizationWithoutXavier,0.179064,"(0, 1)","(-1, 1)",1000,250,0.001,0.0001
3,MatrixFactorizationWithoutXavier,0.179065,"(-1, 1)","(-1, 1)",1000,250,0.001,0.0001
4,MatrixFactorizationWithoutXavier,0.179088,"(-0.2, 0.2)","(0, 0)",1000,250,0.001,0.0001
...,...,...,...,...,...,...,...,...
115,MatrixFactorizationWithXavier,0.180449,xavier,"(0, 1)",300,250,0.001,0.0001
116,MatrixFactorizationWithoutXavier,0.180480,"(0, 1)","(0, 0)",300,220,0.001,0.0001
117,MatrixFactorizationWithoutXavier,0.180510,"(0, 0.2)","(0, 1)",300,260,0.001,0.0001
118,MatrixFactorizationWithoutXavier,0.180538,"(-1, 1)","(0, 0)",300,250,0.001,0.0001


### Top 10 best models

In [37]:
mf_cv_res_pd.head(10)

,model,min_loss,weights,bias,emb_size,epochs,weight_decay,learning_rate
0,MatrixFactorizationWithoutXavier,0.178992,"(0, 0.2)","(0, 0)",1000,250,0.001,0.0001
1,MatrixFactorizationWithoutXavier,0.179021,"(-1, 1)","(0, 1)",1000,250,0.001,0.0001
2,MatrixFactorizationWithoutXavier,0.179064,"(0, 1)","(-1, 1)",1000,250,0.001,0.0001
3,MatrixFactorizationWithoutXavier,0.179065,"(-1, 1)","(-1, 1)",1000,250,0.001,0.0001
4,MatrixFactorizationWithoutXavier,0.179088,"(-0.2, 0.2)","(0, 0)",1000,250,0.001,0.0001
5,MatrixFactorizationWithoutXavier,0.179106,"(-1, 1)","(0, 0)",1000,250,0.001,0.0001
6,MatrixFactorizationWithoutXavier,0.179127,"(0, 0.2)","(-1, 1)",1000,250,0.001,0.0001
7,MatrixFactorizationWithXavier,0.179130,xavier,"(-1, 1)",1000,250,0.001,0.0001
8,MatrixFactorizationWithoutXavier,0.179135,"(-0.2, 0.2)","(-1, 1)",1000,250,0.001,0.0001
9,MatrixFactorizationWithoutXavier,0.179142,"(0, 0.2)","(0, 1)",1000,250,0.001,0.0001


## Let's create logistic regression
Now, to create regression models, both used before and after, we will modify data set in such ways:
1. With modified income column - here the missing values are filled with the mean of the column and modified attr_5_1 columns - here the missing values are filled with the mean of the columns.
2. Without income and attr_5_1 columns.

#### Later we will aply the logic to combine this with MF.

It's going to be used:
1. Before matrix factorization - output of this model will be used as a input to MF.
2. Before matrix factorization - output of MF will be used as input in this model, as one of the features.

We will try 3 diffetent combinations:
1. Regression before MF.
2. Regression after MF.
3. Regression before and after MF.


### Data preprocessing for logistic regression

In [18]:
def fill_with_mean(df):
    """Function fills missing values with mean of the column"""
    col_means = df.mean()
    for index, row in df.iterrows():
        for column in df.columns:
            if column != 'field' and column != 'race_o' and column != 'met_o' and column != 'race' and column != 'goal' and column != 'date' and column != 'go_out' and column != 'career_c':
                if np.isnan(row[column]):
                    df.at[index, column] = col_means[column]
            elif column == 'field' and np.isnan(row['field']):
                df.at[index, column] = 8
            elif column == 'race_o' and np.isnan(row['race_o']):
                df.at[index, column] = 2
            elif column == 'race' and np.isnan(row['race']):
                df.at[index, column] = 2
            elif column == 'goal' and np.isnan(row['goal']):
                df.at[index, column] = 1
            elif column == 'date' and np.isnan(row['date']):
                df.at[index, column] = 6
            elif column == 'go_out' and np.isnan(row['go_out']):
                df.at[index, column] = 2
            elif column == 'career_c' and np.isnan(row['career_c']):
                df.at[index, column] = 2
            elif column == 'met_o' and (np.isnan(row['met_o']) or row['met_o'] != 0):
                df.at[index, column] = 1
    return df

### Option 1
Modified income column - here the missing values are filled with the mean of the column and modified attr_5_1 columns - here the missing values are filled with the mean of the columns.

In [19]:
regression_df_1 = base_df.copy()
regression_df_1 = fill_with_mean(regression_df_1)

### Option 2
Without income and attr_5_1 columns.

In [20]:
regression_df_2 = base_df.copy()
cols_to_drop = ['income', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1']
regression_df_2 = regression_df_2.drop(columns=cols_to_drop)
regression_df_2 = fill_with_mean(regression_df_2)

### Definition of logistic regression

In [21]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        #(out)input_dim is size of our (out)input data
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        pred_y = torch.sigmoid(self.linear(x))
        return torch.squeeze(pred_y, 1)

In [22]:
def LR_train(model, x, y, epochs=100, learning_rate=0.01, weight_decay=1e-5, verbose=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epochs):
        # From numpy to PyTorch tensors.
        # .to(dev) puts code on either gpu or cpu.
        X = torch.FloatTensor(x).to(dev)
        Y = torch.FloatTensor(y).to(dev)
        
        # calls forward method of the model
        y_hat = model(X)
        # Using mean squared errors loss function
        loss = F.mse_loss(y_hat, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if verbose and epoch % 10 == 0: 
            print(loss.item())
            
def LR_test(model, x, y, verbose=False):
    model.eval()
    # .to(dev) puts code on either gpu or cpu.
    X = torch.FloatTensor(x).to(dev)
    Y = torch.FloatTensor(y).to(dev)
    y_hat = model(X)
    # Using mean squared errors loss function
    loss = F.mse_loss(y_hat, Y)
    if verbose:
        print('test loss %.3f ' % loss.item())
    return loss.item()

### Let's find the best model using Cross validation:

In [23]:
epochs_poss = list(range(100, 510, 10))
weight_decay_poss = [0.0,  1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 0.0001, 0.001, 0.01, 0.1, 1.0]
learning_rate_poss = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 0.0001, 0.001, 0.01, 0.1, 1.0] 
#testing every combination of epoch ~ weight_decay ~ learning_rate values
def cross_valid_regression(model, train_x, train_y, test_x, test_y, verbose=False):
    """Function to choose the best hyperparameters for a model."""
    min_loss = np.inf
    best_settings = None

    for (epochs, wd, lr) in tqdm(product(epochs_poss, weight_decay_poss, learning_rate_poss)):
        train(model, train_x, train_y, epochs=epochs, learning_rate=lr, weight_decay=wd)
        test_loss = test(model, test_x, test_y)
        if (test_loss < min_loss) or (test_loss == min_loss and best_settings is not None and epochs < best_settings['epochs']):
            min_loss = test_loss
            best_settings = {'epochs': epochs, 'weight_decay': wd, 'learning_rate_poss': lr}
    if verbose:
        print('min loss %.3f' % min_loss)
        print('best settings are', best_settings)
    return min_loss, best_settings


In [24]:
#train_sample defines size of train set
train_sample = 0.8
#our output is only one number
output_dim = 1

#simple division into training and testing sets
msk = np.random.rand(len(regression_df_1)) < train_sample
train_set = regression_df_1[msk]
test_set = regression_df_1[~msk]

#we want to predict 'dec' basing on other attributes. We dont want 'iid' and 'pid' columns
#to have any impact on results
train_x = train_set.drop(['dec', 'iid', 'pid'], axis=1).values
train_y = train_set['dec'].values 

test_x = test_set.drop(['dec', 'iid', 'pid'], axis=1).values
test_y = test_set['dec'].values 

# input_dim = train_x.shape[1]
# model = LogisticRegression(input_dim, output_dim).to(dev)
# print(cross_valid_regression(model, train_x, train_y, test_x, test_y))

***After trying various different combinations of parameters we have found the best one:***
- epochs: 460
- weight_decay: 0.1
- learning_rate_poss: 0.0001

This combination resulted with loss ~ 0.232


## Preprocessing data before running cross validation to find optimal parameters for training MF for new date decision

In [25]:
regression_df_1.rename(columns={'iid':'id','dec':'decision'},inplace=True)
men_like_women_data_dates = []
women_like_men_data_dates = []

for _, row in regression_df_1.iterrows():
    if row['gender'] == 0:
        # it's a woman
        women_like_men_data_dates.append(row)
    else:
        men_like_women_data_dates.append(row)

men_like_women_df_dates = encode_data(pd.DataFrame(men_like_women_data_dates))
women_like_men_df_dates = encode_data(pd.DataFrame(women_like_men_data_dates))

nr_of_dates = len(men_like_women_df_dates)

print("men_like_women_df:")
print(men_like_women_df_dates)
print("\nwomen_like_men_df:")
print(women_like_men_df_dates)

men_like_women_df:
       id  gender  pid  age_o  race_o  pf_o_att  pf_o_sin  pf_o_int  pf_o_fun  \
100     0     1.0    0   21.0     4.0      15.0      20.0      20.0      15.0   
101     0     1.0    1   24.0     2.0      45.0       5.0      25.0      20.0   
102     0     1.0    2   25.0     2.0      35.0      10.0      35.0      10.0   
103     0     1.0    3   23.0     2.0      20.0      20.0      20.0      20.0   
104     0     1.0    4   21.0     2.0      20.0       5.0      25.0      25.0   
...   ...     ...  ...    ...     ...       ...       ...       ...       ...   
8363  276     1.0  269   26.0     3.0      10.0      10.0      30.0      20.0   
8364  276     1.0  270   24.0     6.0      50.0      20.0      10.0       5.0   
8365  276     1.0  271   29.0     3.0      40.0      10.0      30.0      10.0   
8366  276     1.0  272   22.0     4.0      10.0      25.0      25.0      10.0   
8367  276     1.0  273   22.0     4.0      20.0      20.0      10.0      15.0   

      pf

### Because of the data set being fairly small, we are going to be doing a Leave One Out Cross Validation to get parameters for the Matrix Factorization model training on new date.


**To first train MF and LR models we use parameters computed by cross validation before**

In [112]:
def leave_one_out(MF_df, df_dates_user, df_dates_item, user_id):
    ###Leave one user(of user_id) and every item we have his decision on out of the MF_df
    train_data = []
    test_data = []
    
    for _,row in MF_df.iterrows():
        if row["id"] == user_id:
            test_data.append(row)
        else:
            train_data.append(row)

    ###Leave one user(of user_id) and every item we have his decision on out of the LR_df
    train_set = []
    test_dates = []
    
    for _,row in df_dates_user.iterrows():
        if row["id"] == user_id:
            test_dates.append(row)
        else:
            train_set.append(row)
            
    ### Training dataframe for base MF without user(of user_id), and his decisions upon which we can test the results.
    train_df = pd.DataFrame(train_data)
    test_df = pd.DataFrame(test_data)
    
    ### Training dataframe for base LR without user(of user_id), and his dates upon which we can predict his decision
    ###     and test them against test_df.
    train_set = pd.DataFrame(train_set)
    train_x = pd.concat([train_set, df_dates_item])
    train_y = pd.concat([train_set, df_dates_item])
    
    test_dates = pd.DataFrame(test_dates)
            
    return train_df, test_df, train_x, train_y, test_dates

def new_date_test(MF_model, LR_model, dec_Matrix, df_dates_men, df_dates_women, new_date, nr_men, nr_women, dec,
                 epochs=100, learning_rate=0.001, weight_decay=0.001,K=0.1,P=0.1):
    
    Y = torch.FloatTensor([dec]).to(dev)
    y_hat = get_new_date_dec(MF_model, LR_model, dec_Matrix, df_dates_men, df_dates_women, new_date,nr_men= nr_men,nr_women= nr_women,
                             epochs=100, learning_rate=0.001, weight_decay=0.001,K=0.1,P=0.1,CV_mode=True)
    
    loss = F.mse_loss(y_hat, Y)
    return loss.item()

epochs_poss = list(range(60, 350, 10))
weight_decay_poss = [0.0001, 0.001, 0.01, 0.1, 1.0]
learning_rate_poss = [0.0001, 0.001, 0.01, 0.1, 1.0]
K_poss = [0.05,0.1,0.15]
P_poss = [0.05,0.1]

def LOOCrossValidation(MF_df, df_dates_men , df_dates_women, input_dim = 67, output_dim = 1, nr_men=277, nr_women=274, gender = 1,verbose=False):
    if gender: ###user is a man
        nr_users = nr_men
        nr_items = nr_women
        df_dates_user = df_dates_men
        df_dates_item = df_dates_women
    else:
        nr_users = nr_women
        nr_items = nr_men
        df_dates_user = df_dates_women
        df_dates_item = df_dates_men

    
    min_overall_loss = np.inf
    best_overall_settings = None
    
    for (epochs, wd, lr, K, P) in tqdm(product(epochs_poss, weight_decay_poss, learning_rate_poss, K_poss, P_poss)):

        min_user_loss = np.inf
        best_settings = None
        
        for user_id in range(nr_users):
            
            MF_train_df, dec_test_df, train_x, train_y, dates_test_df = leave_one_out(MF_df, df_dates_user, df_dates_item, user_id)
            
            test_loss = 0.
            (_, dec_row), (_, new_date_row) = [row for row in zip(dec_test_df.iterrows(), dates_test_df.iterrows())][0]
            
            remove_item_id = int(dec_row["pid"])

            _MF_train_df = encode_data(MF_train_df[MF_train_df["pid"] != remove_item_id])
            _train_x = train_x[train_x["pid"] != remove_item_id].drop(['decision', 'id', 'pid'], axis=1).values
            _train_y = train_y[train_y["pid"] != remove_item_id]['decision'].values


            MF_model = MatrixFactorizationWithoutXavier(nr_users, nr_items, weights=(0,0.2), bias=(0,1), emb_size=800).to(dev)
            LR_model = LogisticRegression(input_dim, output_dim).to(dev)

            MF_train(MF_model, _MF_train_df, epochs=100, learning_rate=0.001, weight_decay=0.001)
            LR_train(LR_model, _train_x, _train_y, epochs=460, learning_rate=0.0001, weight_decay=0.1)
            LR_model.eval()
            MF_model.eval()

            if new_date_row["gender"]:
                _df_dates_women = df_dates_men[df_dates_men["id"] != user_id]
                _df_dates_men = df_dates_women[df_dates_women["pid"] != remove_item_id]
                _nr_men = nr_users -1
                _nr_women = nr_items -1
            else:
                _df_dates_women = df_dates_men[df_dates_men["id"] != user_id]
                _df_dates_men = df_dates_women[df_dates_women["pid"] != remove_item_id]
                _nr_men = nr_items -1
                _nr_women = nr_users -1
            dec_Matrix = np.array([[get_mf_dec(MF_model,user_id,item_id) for user_id in range(nr_users-1)]
                                   for item_id in range(nr_items) if item_id != remove_item_id])

            test_loss += new_date_test(MF_model, LR_model, dec_Matrix, _df_dates_men, _df_dates_women, new_date_row,
                                     _nr_men, _nr_women, dec_row["decision"],
                                      epochs=epochs, learning_rate=lr, weight_decay=wd,K=K,P=P)

            test_loss = test_loss / len(dec_test_df["pid"].unique())
            
            if (test_loss < min_user_loss) or (test_loss == min_user_loss and best_user_settings is not None and epochs < best_user_settings['epochs']):
                min_user_loss = test_loss
                best_user_settings = {'epochs': epochs, 'weight_decay': wd, 'learning_rate_poss': lr, '% of similar': K, '% of decisions': P}
                
        if (min_user_loss < min_overall_loss) or (min_user_loss == min_overall_loss and best_overall_settings is not None and epochs < best_overall_settings['epochs']):
                min_overall_loss = min_user_loss
                best_overall_settings = best_user_settings
    if verbose:
        print('min loss %.3f' % min_overall_loss)
        print('best settings are', best_overall_settings)
        print(f"{user_id} loop ran.")

    return min_overall_loss, best_overall_settings

In [ ]:
%%time
min_overall_loss, best_overall_settings = LOOCrossValidation(men_like_women_df,men_like_women_df_dates, women_like_men_df_dates,verbose=True)

0it [00:00, ?it/s]

***Given two vectors x,y where $x_i$ gives user's x decision on partner i (pid = i), computes similiraty measure i.e. on how many positions does vector x match vector y, excluding "-1", the default value for no logistic regression output provided***

In [74]:
def get_similarity(x,y):
    assert len(x) == len(y) , "x should be same size as y"
    _x,_y = [],[]
    for i in range(len(x)):
        if _x != -1 and _y != -1:
            _x.append(x[i])
            _y.append(y[i])
    return np.correlate(np.array(x, dtype=np.float64), np.array(y, dtype=np.float64))[0]

***Given matrix A(which has results only for one gender) where $a_{ij}$ represents output of logistic regression from a date between person(iid = i) and person(pid=j), creates vector C, where each position "i" represents similiarity between input vector person and other person "i" of the same gender***

In [42]:
def split_date(date_row):
    subject_feats = ["gender","age","field","race","imprace","imprelig","income","goal","date","go_out","career_c","sports","tvsports","exercise",
     "dining","museums","art","hiking","gaming","clubbing","reading","tv","theater","movies","concerts","music","shopping",
     "yoga","attr1_1","sinc1_1","intel1_1","fun1_1","amb1_1","shar1_1","attr2_1","sinc2_1","intel2_1","fun2_1","amb2_1",
     "shar2_1","attr3_1","sinc3_1","fun3_1","intel3_1","amb3_1","attr5_1","sinc5_1","intel5_1","fun5_1","amb5_1"]
    subject = date_row[subject_feats]
#     if new_date:
#         partner = date_row.drop(subject_feats)
#         return (0,subject),(0,partner)
    
    partner = date_row.drop(subject_feats + ['decision', 'id', 'pid'])
    return (date_row['id'],subject),(date_row['pid'],partner)

def make_date(subject_info, partner_info):
    return torch.FloatTensor([np.hstack((partner_info,subject_info))]).to(dev)

In [30]:
def get_mf_dec(mf_model, user_id, item_id):
    return mf_model(torch.LongTensor([user_id]).to(dev),torch.LongTensor([item_id]).to(dev)).cpu().detach().numpy()[0]

In [31]:
def update_params(MF_model, weights=(0,0.2),bias=(0,1), user_mode=False):
    if user_mode:
        ###Adding new row to user_emb
        new_emb_row = torch.FloatTensor(1,MF_model.person_emb.embedding_dim).to(dev)
        new_emb_row.data.uniform_(weights[0], weights[1])
        MF_model.person_emb.weight = nn.Parameter(torch.cat((MF_model.person_emb.weight, new_emb_row))).to(dev)

        ###Adding new bias row to user_bias
        new_bias_row = torch.FloatTensor(1,MF_model.person_bias.embedding_dim).to(dev)
        new_bias_row.data.uniform_(bias[0], bias[1])
        MF_model.person_bias.weight = nn.Parameter(torch.cat((MF_model.person_bias.weight, new_bias_row))).to(dev)
    else:
        ###Adding new row to item_emb
        new_emb_row = torch.FloatTensor(1,MF_model.partner_emb.embedding_dim).to(dev)
        new_emb_row.data.uniform_(weights[0], weights[1])
        MF_model.partner_emb.weight = nn.Parameter(torch.cat((MF_model.partner_emb.weight, new_emb_row))).to(dev)

        ###Adding new bias row to item_bias
        new_bias_row = torch.FloatTensor(1,MF_model.parnter_bias.embedding_dim).to(dev)
        new_bias_row.data.uniform_(bias[0], bias[1])
        MF_model.parnter_bias.weight = nn.Parameter(torch.cat((MF_model.parnter_bias.weight, new_bias_row))).to(dev)


def train_one_row(MF_model, df_train,
                  epochs=100, learning_rate=0.001, weight_decay=0.001,weights=(0,0.2),bias=(0,1),user_mode=False):
    ###Creating a mask and registering a hook to zero out gradients of every embedding 
    ###    but the newly added ones, so that mf_model retrains only those.
    item_mask_bias = torch.zeros_like(MF_model.parnter_bias.weight)
    item_bias_hook = MF_model.parnter_bias.weight.register_hook(lambda grad: grad*item_mask_bias)
    
    user_mask_bias = torch.zeros_like(MF_model.person_bias.weight)
    user_bias_hook = MF_model.person_bias.weight.register_hook(lambda grad: grad*user_mask_bias)
    
    item_mask_emb = torch.zeros_like(MF_model.partner_emb.weight)
    item_emb_hook = MF_model.partner_emb.weight.register_hook(lambda grad: grad*item_mask_emb)
    
    user_mask_emb = torch.zeros_like(MF_model.person_emb.weight)
    user_emb_hook = MF_model.person_emb.weight.register_hook(lambda grad: grad*user_mask_emb)
    
    if user_mode:
        ### Setting to train only new_user's parameters
        user_mask_emb[-1] = 1.
        user_mask_bias[-1] = 1.
    else:
        ### Setting to train only new_item's parameters
        item_mask_emb[-1] = 1.
        item_mask_bias[-1] = 1.
    
    MF_model.train()
    MF_train(MF_model,df_train,epochs=100, learning_rate=0.001, weight_decay=0.001)
    MF_model.eval()
    
    ###Remove the hooks
    item_bias_hook.remove()
    item_emb_hook.remove()
    user_bias_hook.remove()
    user_emb_hook.remove()

In [73]:
def get_new_date_dec(mf_model, 
                     LR_model,
                     dec_Matrix,
                     df_dates_men,
                     df_dates_women,
                     new_date = pd.Series,
                     K = 0.1,
                     P = 0.1,
                     nr_men = 277, nr_women = 274,
                     epochs=100, learning_rate=0.001, weight_decay=0.001,weights=(0,0.2),bias=(0,1),
                     CV_mode = False
                    ):
    
    MF_model = copy.deepcopy(mf_model).to(dev)
    user,item = split_date(new_date)
    ### Determine number of users and items depending on target's gender
    if new_date["gender"]: 
        #target is a man
        nr_users = nr_men
        nr_items = nr_women
        df_user_dates = df_dates_men
        df_item_dates = df_dates_women
    else:
        #target is a woman
        nr_users = nr_women
        nr_items = nr_men
        df_user_dates = df_dates_women
        df_item_dates = df_dates_men

    

    ###---------------------------------------------------Add new item----------------------------------------------------###
    
    ###First get every user's decision on the new item using logistic regression and -1 when user's date_info not provided
    new_item_dec = np.array([-1.] * nr_users, dtype= np.float64)
    for _,row in df_user_dates.iterrows():
        _user,_ = split_date(row)
        _date = make_date(_user[1],item[1])
        log_reg_output = LR_model(_date)
        new_item_dec[int(_user[0])-1] = log_reg_output

    ### Compute similarity rate by comparing new item's decisions(output of logistic regression)
    ### with every other item's decision(output of matrix factorization)
    similarity_vec = [get_similarity(new_item_dec, item_dec) for item_dec in dec_Matrix] ## change get_similarity(*) 
    
    ### Get ids of K most similar items
    most_sim_idx = np.argsort(similarity_vec)[:int(nr_items * K)]
    
    ###Compute the average decisions from K most similar items.
    avg_dec_vec = np.mean(dec_Matrix[most_sim_idx],axis=0)
    
    ###Pick nr_users*P(percantage of users) of these decisions and use them as baseline decisions to retrain new item's decisions using MatrixFactorization
    new_item_dec = np.array([-1.] * nr_users, dtype= np.float64)
    random_idx = np.random.choice(range(nr_users),int(nr_users * P))
    new_item_dec[random_idx] = avg_dec_vec[random_idx]
   
    ###Prepare dataframe for MF_model to train on
    df_train = pd.DataFrame(data=[[user_id,nr_items,dec] for user_id,dec in enumerate(new_item_dec) if dec != -1] ,columns=["id", "pid", "decision"])
    
    ###Update MF_model parameters to train only the new item's row
    update_params(MF_model)
    train_one_row(MF_model,df_train)

    ###Add new item's decisions to the dec_Matrix
    new_item_dec = np.array([get_mf_dec(MF_model, user_id, nr_items) for user_id in range(nr_users)],dtype=np.float64)
    updated_dec_Matrix = np.vstack((dec_Matrix,new_item_dec))
    ###New item has been added.
    nr_items += 1
    
    
    ###---------------------------------------------------Add new user----------------------------------------------------###
    
    ###First get every new user's decisions on every item using logistic regression and -1 when item's date_info not provided
    new_user_dec = np.array([-1.] * nr_items, dtype= np.float64)
    for idx,row in df_item_dates.iterrows():
        _,_item = split_date(row)
        _date = make_date(user[1],_item[1])
        log_reg_output = LR_model(_date)
        new_user_dec[int(_item[0])-1] = log_reg_output

    ###Then get id's of most similar users to the new one, by comparing their decisions we got 
    ###    using matrix factorization and new items logistic regression ones.
    similarity_vec = [get_similarity(new_user_dec, np.array([item_dec[idx] for item_dec in updated_dec_Matrix],dtype=np.float64)) for idx in range(nr_items)]
    most_sim_idx = np.argsort(similarity_vec)[:int(nr_items * K)]
    
    ###Compute the average decisions from K most similar users.
    avg_dec_vec = np.mean([[item_dec[idx] for item_dec in updated_dec_Matrix] for idx in most_sim_idx],axis=0)
    
    ###Take nr_items*P(percantage of items number) of these decisions and use them as a baseline for MF.
    new_user_dec = np.array([-1.] * nr_items, dtype= np.float64)
    random_idx = np.random.choice(range(nr_items-1),int(nr_items * P))
    new_user_dec[random_idx] = avg_dec_vec[random_idx]
    
    ###Prepare dataframe for MF_model to train on
    df_train = pd.DataFrame(data=[[nr_users,item_id,dec] for item_id, dec in enumerate(new_user_dec) if dec != -1],columns=["id", "pid", "decision"])

    ###Update MF_model parameters to train only the new user's row
    update_params(MF_model,user_mode=True)
    train_one_row(MF_model,df_train,user_mode=True)
    
    if CV_mode:
        return MF_model(torch.LongTensor([nr_users]).to(dev),torch.LongTensor([nr_items-1]).to(dev))
    
    ###return new user's decision on new item
    return get_mf_dec(MF_model, nr_users, nr_items-1)